In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = pd.read_csv('clean_housing.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

# Build a Baseline Model
In order to fit a baseline model for use in comparisons with the more complete model I will conduct these steps:
1. Handle Muliticolinearity
2. Standardize Variables
3. Fit and Validate the Baseline Model

## 1. Handle Multicolinearity 

In [4]:
high_corr = ((abs(data.corr())> .8).sum()>1)
high_corr

price              False
bedrooms           False
bathrooms          False
sqft_living         True
floors             False
view               False
grade              False
sqft_above          True
sqft_basement      False
lat                False
long               False
sqft_living15      False
WaterFront         False
month_sold         False
age                 True
yrs_reno            True
Renovated          False
dist_to_Seattle    False
Bellevue           False
Federal Way        False
Kent               False
Renton             False
Seattle            False
rel_size           False
dtype: bool

In [5]:
data[['sqft_living', 'sqft_above', 'age', 'yrs_reno']].corr()

,sqft_living,sqft_above,age,yrs_reno
sqft_living,1.000000,0.839924,-0.366368,-0.376163
sqft_above,0.839924,1.000000,-0.483546,-0.482755
age,-0.366368,-0.483546,1.000000,0.938759
yrs_reno,-0.376163,-0.482755,0.938759,1.000000


To avoid multicolinearity issues I will drop sqft_above, because the information already exists in a combination of sqft_living and sqft_basement. 

In [ ]:
data.drop('sqft_above',)

## 2. Standardize Variables 
To standardize the data I'll be using scikit learn's Robust Scalar function because of the existence of significant outliers in the independent variable, price.

In [ ]:
data['price'].hist();
data.drop('id', axis=1, inplace=True)

In [ ]:
#Separate data into categorical and continuous groups. 
cat_data = data.select_dtypes(include='uint8')

con_data = (data.select_dtypes(exclude='uint8')
#Keep certain categorical data separate to use for visuals.
            .drop(['Renovated?', 'month_sold', 'nearest_city', 
                  'waterfront'], axis=1))
groups = data[['Renovated?', 'month_sold', 'nearest_city',
               'waterfront']]
con_data.head()

In [ ]:
def scale(col):
    return (con_data[col]
            -con_data[col].mean())/con_data[col].std()

In [ ]:
#Scale the continuous data. 
scaled_con_data = pd.DataFrame([])
for col in con_data.columns:
    scaled_con_data[col] = scale(col) 
scaled_con_data.describe()

In [ ]:
#Join continuous and categorical data. 
model_data = scaled_con_data.join(cat_data, how='outer')
model_data.head()

#Save complete dataset for visualising later.
data_fin = model_data.join(groups, how='outer')

## Filter by P-values

# Build a Baseline Model

In [ ]:
X = model_data.drop('price', axis=1)
y = model_data['price']
y.describe()

In [ ]:
(X_train,X_test,
 y_train,y_test)=train_test_split(X,y,test_size=.2,
                                  random_state=37)

In [ ]:
linreg = LinearRegression()
model1 = linreg.fit(X_train,y_train)

In [ ]:
def report(model, ind_train, ind_test):
    """
    Print relevant statistics for a model.
    
    Parameters:
    model: Fitted LinearRegression object
    ind_train: independent variables for training set
    ind_test: independent variables for test set
    """
    pred_y_train = model.predict(ind_train)
    pred_y_test = model.predict(ind_test)
    
    #Print top 5% of variables by size of coefficient.
    coefs = []
    high_coefs = []
    for i in range(0, len(model.coef_)):
        coefs.append((model.coef_[i],ind_train.columns[i]))
    for coef in coefs:
        if abs(coef[0]) > abs(np.quantile(model.coef_,.95)):
            high_coefs.append(coef)
    print('High Impact Variables:\n')
    for variable in high_coefs:
        print('Variable: {}\nCoefficient: {}\n'
              .format(variable[1],variable[0]))

    #Print MSE for the train an test set.
    train_mse = mean_squared_error(y_train, pred_y_train)
    test_mse = mean_squared_error(y_test, pred_y_test)
    print('\nTrain MSE: {}\nTest MSE: {}\nDifference:{}\n'
          .format((train_mse),(test_mse),
                  (train_mse-test_mse)))
    
    #Print R^2 against the test data. 
    print('Train R^2: {}'
          .format((r2_score(y_train,pred_y_train))))
    print('Test R^2: {}\n'
          .format((r2_score(y_test,pred_y_test))))
    

    plotdf = pd.DataFrame([])
    plotdf['test_resids'] = pred_y_test-y_test
    plotdf['y_test'] = y_test
    sns.jointplot(x = 'y_test', y = 'test_resids',
                  data=plotdf, kind='kde')
    plt.show();
    
    plt.scatter(pred_y_test, y_test)
    plt.plot(pred_y_test, pred_y_test, color='black', 
             label='Predicted Price')
    plt.legend();

In [ ]:
report(model1, X_train, X_test)

In [ ]:
pred_y = model1.predict(X)

In [ ]:
X['resids']= pred_y - y
X['price']=y
X.loc[abs(X['resids'])>2].describe()

In [ ]:
X.describe()

Based on the MSE and R-squared, the model seems fairly accurate. However, based on the KDE plot of residuals, it is clear that the model could be more generalizable, especially for high value homes. Polynomial relationships may be causing drift. 

# Train the Model
In order to train the model and verify it's validity I will conduct the following steps:

1. Find and include interaction features.
2. Find and include polynomial features.
3. Satisfy Assumptions.
4. Validate Model

During this process I will continually test the models and eliminate variables that do not serve the model.

## Find and Include Interaction Features

In [ ]:
def plot_interaction(col1, col2):
    """
    Plot the regression lines of variables grouped by
    high and low values. Non-parellel lines show 
    interaction of variables.
    
    Parameters:
    col1: pandas Series. Variable to group by.
    col2: pandas Series. Variable to plot by
    """
    sample = X_train.join(y_train, how='outer')
    
    hisample = (sample.loc[sample[col1]
                           >sample[col1].quantile(.5)])
    losample = (sample.loc[sample[col1]
                           <sample[col1].quantile(.5)])
    
    sns.regplot(x=col2, y='price', data=hisample, 
                scatter=False, truncate=True, 
                label='High Values of {}'.format(col1))
    sns.regplot(x=col2, y='price', data=losample, 
                scatter=False, 
                label='Low Values of {}'.format(col1))
    plt.title('Interaction of {} and {}'.format(col1, col2))
    plt.legend()
    plt.show();
    print('\n*********************\n')

In [ ]:
def find_interactions(n, model, ind_train):
    """
    Returns n most predictive interactions based on low MSE.
    
    Parameters:
    n: int. the number of interactions selected.
    model: LinearRegression() object being tested. 
    ind_train: the independent variables in the training set.
    """
    combos = list(combinations(ind_train.columns, 2))
    print('Testing {} combinations.\n'.format(len(combos)))
    inters = [(100,0,0)]*n
    temp_X = ind_train.loc[:]
    for combo in combos:
        temp_X['interaction']=(ind_train.loc[:, combo[0]]
                               *ind_train.loc[:, combo[1]])
        linreg = LinearRegression()
        model = linreg.fit(temp_X, y_train)
        y_pred = model.predict(temp_X)
        score = round(mean_squared_error(y_train, y_pred),3)
        if score < inters[-1][0]:
            inters.append((score, combo[0], combo[1]))
            inters = sorted(inters, reverse=False)[:n]
    for inter in inters:
        print('MSE including interaction of {} and {}: {}'
              .format(inter[1], inter[2], inter[0]))
        plot_interaction(inter[1], inter[2])
    fin_inters = [i[1:] for i in inters]
    return fin_inters

In [ ]:
interactions = find_interactions(20, model1, X_train)

It's clear that a home's relationship to Seattle is important to this analysis. In order to account for this I will first remove any interactions that showed parellell relationships in the plots or are redundant. In the case of redundancies I will use the interaction of distance rather than city so as to not overly weight any city. 

I will then create a model that includes six of the most impactful interactions. See complete description of why I reasoned that each interaction warranted inclusion in the README.

In [ ]:
print('Homes south of Seattle: {}'
      .format(data[data['lat']<47.608013]['lat'].count()))
print('Distance of Farthest Southern home: {}'
      .format(47.608013-data['lat'].min()))
print('Distance of Farthest Nothern home: {}'
      .format(data['lat'].max()-47.608013))
data['lat'].describe()

In [ ]:
bad = [('grade', 'Seattle'),('grade', 'lat'),
       ('sqft_living', 'Seattle'),('sqft_living', 'lat'),
       ('grade', 'Kent'),('long', 'Seattle'),
       ('lat', 'Seattle'),('dist_to_Seattle', 'Bellevue'),
       ('dist_to_Seattle', 'Kent'),('lat', 'Kent'),
       ('sqft_living', 'Kent'),('sqft_living15', 'Kent'),
       ('sqft_living15', 'Seattle'),
       ('bathrooms', 'dist_to_Seattle')]
for inter in bad:
    interactions.remove(inter)

In [ ]:
def add_interactions(interactions, ind_train, ind_test):
    """
    Use forward selection based on lowest MSE to select
    and add most predictive interactions to a new model.
    
    Parameters:
    interactions: list of tuples outputed by find_interaction
    function.
    ind_train: independent variables training data.
    ind_test: independent variables test data.
    
    Returns:
    (new_model, new_x_with_interactions)
    """
    additions = interactions
    X_temp_tr = ind_train.loc[:]
    X_best_tr = ind_train.loc[:]
    X_best_t = ind_test.loc[:]
    scores = []
    baseline = 100
    while additions:
        for inter in additions:
            X_temp_tr[inter[0]
                      +' * '
                      +inter[1]]=(X_temp_tr.loc[:, inter[0]]
                                  *X_temp_tr.loc[:, inter[1]])
            linreg = LinearRegression()
            model = linreg.fit(X_temp_tr, y_train)
            y_pred = model.predict(X_temp_tr)
            score = round(mean_squared_error(y_train,
                                             y_pred),5)
            scores.append((score, inter[0], inter[1]))
            best = sorted(scores)[0]
            x_temp_tr = X_best_tr.loc[:]
        scores = []
        if best[0] <= baseline:
            additions.remove(best[1:])
            baseline = best[0]
            X_best_tr[best[1]
                      +' * '
                      +best[2]]=(X_temp_tr.loc[:, best[1]]
                                 * X_temp_tr
                                 .loc[:, best[2]])
            X_best_t[best[1]
                     +' * '
                     +best[2]]=(X_best_t.loc[:, best[1]]
                                *X_best_t.loc[:, best[2]])
            linreg = LinearRegression()
            model = linreg.fit(X_best_tr, y_train)
            y_pred = model.predict(X_best_tr)
            t_pred = model.predict(X_best_t)
            print('Interaction Added: {} * {}'
                  .format(best[1], best[2]))
            print('Current Train MSE: {}'
                  .format(best[0]))
            print('Current Test MSE: {}'
                  .format(round(mean_squared_error
                                (y_test, t_pred),5)))
            print('Current Difference: {}\n'
                  .format(round(best[0]
                          -round(mean_squared_error
                                 (y_test, t_pred),5),5)))
        else:
            print('complete')
            break
    linreg = LinearRegression()
    new_model = linreg.fit(X_best_tr, y_train)
    return(new_model, X_best_tr, X_best_t)

In [ ]:
model2, X_train2, X_test2 = add_interactions(interactions, 
                                             X_train, X_test)

In [ ]:
report(model2, X_train2, X_test2 )

## Find and include Polynomial Features
To find polynomial features I will iterate through the data, attempting to find data that suggests a polynomial relationship between price and the independent variable. I will then   

In [ ]:
train_set = X_train2.join(y_train, how='outer')

In [ ]:
def get_polynomial_features():
    features = []
    for col in X_train2.columns:
        scores = []
        for degree in range(1,10):
            df = pd.DataFrame(X_train2[col])
            poly = PolynomialFeatures(degree)
            X_poly_train = poly.fit_transform(df)
            reg_poly = LinearRegression().fit(X_poly_train,
                                              y_train)
            y_pred = reg_poly.predict(X_poly_train)
            score = round(mean_squared_error(y_train,
                                             y_pred),5)
            scores.append((score, degree, col))
            best_score = sorted(scores)[0] 
        if best_score[1] > 1:
            print('Variable: {} should be factored by {}'
                  .format(best_score[2], 
                          best_score[1]))
            features.append(best_score)
    return features 

In [ ]:
poly_feat = get_polynomial_features()

In [ ]:
X_train2 = X_train2.reset_index()
X_train2.drop('index', axis=1, inplace=True)

In [ ]:
X_test2 = X_test2.reset_index()
X_test2.drop('index', axis=1, inplace=True)

In [ ]:
X_train3 = X_train2.loc[:]
X_test3 = X_test2.loc[:]
for feat in poly_feat: 
    X_train3[feat[2]+'^2'] = X_train3[feat[2]]**feat[1]
    X_test3[feat[2]+'^2'] = X_test3[feat[2]]**feat[1]
    
linreg = LinearRegression()
model3 = linreg.fit(X_train3, y_train)

In [ ]:
report(model3, X_train3, X_test3)

In [ ]:
X_train3.head()

In [ ]:
train = pd.DataFrame(X_train2[feat[2]])
    X_poly_train = poly.fit_transform(train)
    
    test = pd.DataFrame(X_test2[feat[2]])
    X_poly_test = poly.fit_transform(test)

In [ ]:
def optimize_variables(X,y):
    """
    Forward selects variables by adjusted R^2.
    
    Parameters:
    X: pandas dataframe. independent variables.
    y: pandas series. dependent variable.
    
    Returns:
    Optimized model. 
    """
current_score, best_score = 0, 0
remaining = set(X.columns)
selected = pd.DataFrame()
while remaining and not current_score > best_score:
    scores_with_candidates = []
    for candidate in remaining:
        test_x = selected.join(X[str(candidate)], how='outer')
        score = sm.OLS(y,test_x).fit().rsquared_adj
        scores_with_candidates.append((score,candidate))
    scores_with_candidates.sort()
    best_score, best_candidate = scores_with_candidates.pop()
    if current_score < best_score:
        remaining.remove(best_candidate)
        selected = selected.join(X[str(best_candidate)], how='outer')
        current_score = best_score
print('Selected Columns: {}'.format(selected.columns))
best_vars = sm.add_constant(selected)
return best_vars

In [ ]:
def isolate(col):
    ind_var = pd.DataFrame(X[col]) 
    frozen_var = X.drop(col, axis=1)
    for fro_col in frozen_var.columns:
        frozen_var[fro_col] = frozen_var[fro_col].mean()
    iso_X = ind_var.join(frozen_var, how='outer')
    return iso_X